In [1]:
import torch
from pytorch_transformers import *
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from torch import nn
import random
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import run_squad as rs

ModuleNotFoundError: No module named 'pytorch_transformers'

In [2]:
MODELS = [(BertForQuestionAnswering,       BertTokenizer,      'bert-base-uncased')]

NameError: name 'BertForQuestionAnswering' is not defined

In [3]:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    config = BertConfig.from_pretrained(pretrained_weights)


In [4]:
class RL_optimizer(BertModel):
    def __init__(self, config):
        super(RL_optimizer, self).__init__(config)
        self.reward = 0
        self.bert = BertModel(config)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None,
                position_ids=None, head_mask=None):
        
        outputs = self.bert(input_ids, position_ids=position_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, head_mask=head_mask)
        
        return outputs

In [5]:
rl = RL_optimizer(config)

In [6]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in rl.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in rl.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=0.9)



In [7]:
input_ids = torch.tensor(tokenizer.encode('Who is Beyonce?')).unsqueeze(0)
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])
# start_positions = None
# end_positions = None

In [8]:
# outputs = model(input_ids, start_positions=start_positions, end_positions=end_positions)
outputs = model(input_ids)

In [9]:
outputs

(tensor([[-0.2882, -0.1364, -0.2691, -0.1777]], grad_fn=<SqueezeBackward1>),
 tensor([[-0.0256, -0.2333, -0.2364, -0.1637]], grad_fn=<SqueezeBackward1>))

In [10]:
def train():
    rl.zero_grad()
    rl.train()
    op2 = rl(input_ids, start_positions=start_positions, end_positions=end_positions)
    loss = op2[0]
    r = calc_rewards()
    print(f'reward = ', r)
    mean_loss = loss + r
    mean_loss.backward()

In [11]:
def update_weights():
    optimizer.step()

In [12]:
def calc_rewards():
    """For now, we're using random rewards to check if the flow works fine"""
    return random.choice([-0.5, 0.5])

In [13]:
st = outputs[1]
# end = outputs[2]

In [14]:
st[0][0]

tensor(-0.0256, grad_fn=<SelectBackward>)

In [15]:
tokenizer.decode(st.flatten().tolist())

'[UNK] [UNK] [UNK] [UNK]'

In [16]:
tokenizer.decode(st[0])

'[UNK] [UNK] [UNK] [UNK]'

In [17]:
input_ids = torch.tensor(tokenizer.encode('Who is Beyonce?')).unsqueeze(0)
input_ids

tensor([[ 2040,  2003, 20773,  1029]])

In [18]:
tokenizer.decode(input_ids.flatten().tolist())

'who is beyonce?'

In [19]:
st.flatten().tolist()

[-0.025617510080337524,
 -0.23332782089710236,
 -0.23638038337230682,
 -0.16365335881710052]

In [20]:
train_dataset = '/Users/avaamo/workspace/question_gen/bloomsburyai/question-generation/data/train-v1.1.json'
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=8)

In [21]:
train_iterator = trange(int(10), desc="Epoch", disable=-1 not in [-1, 0])
model.zero_grad()

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=-1 not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
            print(batch)
            batch = tuple(t.to('cpu') for t in batch)
            print(batch)
            break


Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

['i', 'o', '/', 'e', 'u', 'e', 'a', 'n']


AttributeError: 'str' object has no attribute 'to'

In [38]:
class args_list:
    def __init__(self):
        self.local_rank = -1
        self.gpu = -1
        self.train_file = '/Users/avaamo/workspace/question_gen/bloomsburyai/question-generation/data/train-v1.1.json'
        self.evaluate = 0
        self.predict_file = False
        self.model_type = 'bert'
        self.model_name_or_path = 'bert-base-uncased'
        self.output_dir = './outputs/'
        self.tokenizer_name = 'BertTokenizer'
        self.max_seq_length = 384
        self.version_2_with_negative = False
        self.doc_stride = 128
        self.max_query_length = 64
        

In [39]:
args = args_list()

In [ ]:
dataset = rs.load_and_cache_examples(args, tokenizer)